<a href="https://colab.research.google.com/github/Fizza-Rubab/CS432-GPU/blob/main/HWs/assignment1_fr06161.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1
## Fizza Rubab

### Question 1
The first task asks you to write CUDA code to initialize a random array of 1000000 elements.
Provide the following:

#### Part (1)
Share the amount of time it takes to populate data in the array on the CPU. Use any
timing function like clock() or hiresolution counters to note the amount of time it takes to
generate data on the CPU. (+20)


In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-8wxps8_n
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-8wxps8_n
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=9e97d9052f5d73573552a018d4f63d57627fd9f671eeba5cfa20b9040f1042f7
  Stored in directory: /tmp/pip-ephem-wheel-cache-74z8kcu7/wheels/f3/08/cc/e2b5b0e1c92df07dbb50a6f024a68ce090f5e7b2316b41756d
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [2]:
!nvidia-smi

Thu Jan 26 15:06:14 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0    29W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
%%cu
#include <bits/stdc++.h>
#include <stdio.h>

inline cudaError_t checkCudaErr(cudaError_t err, const char* msg) {
  if (err != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg, cudaGetErrorString(err));
  }
  return err;
}
 

__global__ void MyKernel(int* data) 
{
    
}

int main()
{
  const int N = 1000000;
  clock_t start, end;
  start = clock();
  int array[N];
  for (int i=0; i<N; i++)
    array[i] = rand();
  end = clock();
  double time_elapsed = double(end - start) / double(CLOCKS_PER_SEC);
  printf("Time taken to generate a 1000000 element random array is %f seconds.\n", time_elapsed);
  printf("Displaying first 10 elements for verification:\n");
  for (int i = 0; i < 10; i++)
      printf("%d\n", array[i]);
  return 0;
}


Time taken to generate a 1000000 element random array is 0.008089 seconds.
Displaying first 10 elements for verification:
1804289383
846930886
1681692777
1714636915
1957747793
424238335
719885386
1649760492
596516649
1189641421



#### References
1. https://www.geeksforgeeks.org/measure-execution-time-with-high-precision-in-c-c/
2. https://www.tutorialspoint.com/c_standard_library/c_function_rand.htm

### Part(2)
Find a way to do the random data initialization of the array on the GPU. Do not just
copy data from CPU array to GPU. Instead, you should use any library like cuRand but
you should provide relevant references. Note the time it takes to run the kernel using the
CUDA event API to calculate the amount of time needed by the GPU. (+20)

In [52]:
%%writefile /content/q1part2.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <curand_kernel.h>
inline cudaError_t checkCudaErr(cudaError_t err, const char* msg) {
  if (err != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg, cudaGetErrorString(err));
  }
  return err;
}

__global__ void device_api_kernel(curandState *states, int *result, int N)
{
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    curandState *state = states + i;
    curand_init(9384, i, 0, state);
    result[i] = curand(state);
}


int main(int argc, char **argv)
{
    int N = 1000000;
    curandState *states = NULL;
    int *d_data, *h_data;
	  int threads_per_block =  64;
	  int blocks_per_grid = ceil(N/threads_per_block + 1);
    cudaMalloc((void **)&d_data, sizeof(int) * N);
    cudaMalloc((void **)&states, sizeof(curandState)*threads_per_block * blocks_per_grid);
    h_data = (int *)malloc(sizeof(int) * N);

    cudaEvent_t start, stop;
    float elapsedTime;

    cudaEventCreate(&start);
    cudaEventRecord(start,0);


    device_api_kernel<<<blocks_per_grid, threads_per_block>>>(states, d_data, N);


    cudaEventCreate(&stop);
    cudaEventRecord(stop,0);
    cudaEventSynchronize(stop);

    cudaEventElapsedTime(&elapsedTime, start,stop);
    printf("Elapsed time for generating random numbers: %fs\n" ,elapsedTime/1000);

    checkCudaErr(cudaMemcpy(h_data, d_data, sizeof(int) * N, cudaMemcpyDeviceToHost), "memcopy");
    printf("Displaying first 10 random numbers from array of 1000000 elements\n");
    for (int i = 0; i < 10; i++)
        printf("%d\n", h_data[i]);
    free(h_data);
    cudaFree(d_data);
    cudaFree(states);
    return 0;
}

Overwriting /content/q1part2.cu


In [53]:
!nvcc /content/q1part2.cu -lcurand -o q1part2 && ./q1part2

Elapsed time for generating random numbers: 0.044527s
Displaying first 10 random numbers from array of 1000000 elements
-456389480
-1997568819
-948239084
1671272147
713009428
1077169352
-1363832762
661140645
387525661
436233125


### Part(3)
Copy the random array data generated on the GPU from the GPU to the CPU using
cudaMemcpy. Note the amount of time it takes to move the data from GPU to the CPU.
(+10)

In [1]:
%%writefile /content/q1part3.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <curand_kernel.h>
inline cudaError_t checkCudaErr(cudaError_t err, const char* msg) {
  if (err != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg, cudaGetErrorString(err));
  }
  return err;
}

__global__ void device_api_kernel(curandState *states, int *result, int N)
{
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    curandState *state = states + i;
    curand_init(9384, i, 0, state);
    result[i] = curand(state);
}



int main(int argc, char **argv)
{
    int N = 1000000;
    curandState *states = NULL;
    int *d_data, *h_data;
	  int threads_per_block =  64;
	  int blocks_per_grid = ceilf(N/threads_per_block + 1);
    cudaMalloc((void **)&d_data, sizeof(int) * N);
    cudaMalloc((void **)&states, sizeof(curandState)*threads_per_block * blocks_per_grid);
    h_data = (int *)malloc(sizeof(int) * N);

    device_api_kernel<<<blocks_per_grid, threads_per_block>>>(states, d_data, N);
    cudaEvent_t start, stop;
    float elapsedTime;
    cudaEventCreate(&start);
    cudaEventRecord(start,0);
    checkCudaErr(cudaMemcpy(h_data, d_data, sizeof(int) * N, cudaMemcpyDeviceToHost), "memcopy");
    cudaEventCreate(&stop);
    cudaEventRecord(stop,0);
    cudaEventSynchronize(stop);

    cudaEventElapsedTime(&elapsedTime, start,stop);
    printf("Elapsed time for Copying data from device to host: %fs\n" ,elapsedTime/1000);
    printf("Displaying first 10 random numbers from array of 1000000 elements\n");
    for (int i = 0; i < 10; i++)
        printf("%d\n", h_data[i]);
    free(h_data);
    cudaFree(d_data);
    cudaFree(states);
    return 0;
}

Writing /content/q1part3.cu


In [2]:
!nvcc /content/q1part3.cu -lcurand -o q1part3 && ./q1part3

CUDA Runtime error at memcopy: no CUDA-capable device is detected
Elapsed time for Copying data from device to host: 0.000000s
Displaying first 10 random numbers from array of 1000000 elements
0
0
0
0
0
0
0
0
0
0


## Question 2
Write CUDA code to calculate the sum of 1000 elements array and output the sum. Calculate
the total time for doing the calculation on the CPU as well as on the GPU. Use an appropriate
execution configuration when you launch your kernel. Share the following information from your
solution.

1. Code compiles and outputs as expected both on CPU and GPU. (+10)
2. Plot your results in a graph and show both your results (CPU/GPU) against different
dataset sizes (1000,10000,100000) and different execution configurations (+30)
3. Do proper error handling in your code so any CUDA error should be reported to the user.
(+10)



In [81]:
%%writefile /content/q2part1.cu
#include <stdio.h>
#include <bits/stdc++.h>

inline cudaError_t checkCudaErr(cudaError_t err, const char* msg) {
  if (err != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg, cudaGetErrorString(err));
  }
  return err;
}
 

__global__ void InitData(int* data, int* d_sum, int N) 
{

    // Since only one thread, threadIdx.x = 0, just use i = 0
    for (int i=threadIdx.x;i<N;i++)
      *d_sum+=data[i]; //Store sum
}

int main()
{
  const int N = 100;  

  int* h_data = (int*)calloc(N, sizeof(int));
  for (int j=0;j<N;j++)
    h_data[j]=1;
  
  clock_t start, end;
  start = clock();
  int h_sum = 0;
  for (int j=0;j<N;j++)
    h_sum+=h_data[j];
  end = clock();
  double time_elapsed = double(end - start) / double(CLOCKS_PER_SEC);
  printf("Sum of the array computed at CPU is %d\nTotal Time taken on CPU: %fs\n", h_sum, time_elapsed);

    
  int* d_data;
  int* d_sum;
  //allocate memory on device
  checkCudaErr(cudaMalloc((void **)&d_data, N*sizeof(int)), "cudaMalloc");
  checkCudaErr(cudaMalloc((void **)&d_sum, sizeof(int)), "cudaMalloc");

  //copy device data to host memory
  checkCudaErr(cudaMemcpy(d_data, h_data, N*sizeof(int), cudaMemcpyHostToDevice), "cudaMemcpy D->H");
  
  cudaEvent_t start_gpu, stop_gpu;
  float time_elapsed_gpu;
  cudaEventCreate(&start_gpu);
  cudaEventRecord(start_gpu,0);
  //call kernel  
  InitData<<<1,1>>>(d_data, d_sum, N);

  cudaEventCreate(&stop_gpu);
  cudaEventRecord(stop_gpu,0);
  cudaEventSynchronize(stop_gpu);

  cudaEventElapsedTime(&time_elapsed_gpu, start_gpu, stop_gpu);

  checkCudaErr(cudaDeviceSynchronize(), "Syncronization");
  checkCudaErr(cudaGetLastError(), "GPU Error");
 
  //copy device data to host memory
  checkCudaErr(cudaMemcpy(h_data, d_data, N*sizeof(int), cudaMemcpyDeviceToHost), "cudaMemcpy D->H");
  checkCudaErr(cudaMemcpy(&h_sum, d_sum, sizeof(int), cudaMemcpyDeviceToHost), "cudaMemcpy D->H");
  
  printf("Sum of the array computed in GPU is %d\nTotal Time taken on GPU: %fs\n", h_sum, time_elapsed_gpu/1000);

  //release GPU memory
  cudaFree(d_data);
  cudaFree(d_sum);
  free(h_data);

  return 0;
}

Overwriting /content/q2part1.cu


In [82]:
!nvcc /content/q2part1.cu -lcurand -o q2part1 && ./q2part1

Sum of the array computed at CPU is 100
Total Time taken on CPU: 0.000002s
Sum of the array computed in GPU is 100
Total Time taken on GPU: 0.000029s


In [89]:
%%writefile /content/q2part1.cu
#include <stdio.h>
#include <bits/stdc++.h>

inline cudaError_t checkCudaErr(cudaError_t err, const char* msg) {
  if (err != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg, cudaGetErrorString(err));
  }
  return err;
}
 

__global__ void InitData(int* data, int* d_sum, int N) 
{

    // Since only one thread, threadIdx.x = 0, just use i = 0
    for (int i=0;i<N;i++)
      *d_sum+=data[i]; //Store sum
}

int main()
{
  const int N = 100;  

  int* h_data = (int*)calloc(N, sizeof(int));
  for (int j=0;j<N;j++)
    h_data[j]=1;
  
  clock_t start, end;
  start = clock();
  int h_sum = 0;
  for (int j=0;j<N;j++)
    h_sum+=h_data[j];
  end = clock();
  double time_elapsed = double(end - start) / double(CLOCKS_PER_SEC);
  printf("Sum of the array computed at CPU is %d\nTotal Time taken on CPU: %fs\n", h_sum, time_elapsed);

    
  int* d_data;
  int* d_sum;
  //allocate memory on device
  checkCudaErr(cudaMalloc((void **)&d_data, N*sizeof(int)), "cudaMalloc");
  checkCudaErr(cudaMalloc((void **)&d_sum, sizeof(int)), "cudaMalloc");

  //copy device data to host memory
  checkCudaErr(cudaMemcpy(d_data, h_data, N*sizeof(int), cudaMemcpyHostToDevice), "cudaMemcpy D->H");
  
  cudaEvent_t start_gpu, stop_gpu;
  float time_elapsed_gpu;
  cudaEventCreate(&start_gpu);
  cudaEventRecord(start_gpu,0);
  //call kernel  
  InitData<<<1,10>>>(d_data, d_sum, N);

  cudaEventCreate(&stop_gpu);
  cudaEventRecord(stop_gpu,0);
  cudaEventSynchronize(stop_gpu);

  cudaEventElapsedTime(&time_elapsed_gpu, start_gpu, stop_gpu);

  checkCudaErr(cudaDeviceSynchronize(), "Syncronization");
  checkCudaErr(cudaGetLastError(), "GPU Error");
 
  //copy device data to host memory
  checkCudaErr(cudaMemcpy(h_data, d_data, N*sizeof(int), cudaMemcpyDeviceToHost), "cudaMemcpy D->H");
  checkCudaErr(cudaMemcpy(&h_sum, d_sum, sizeof(int), cudaMemcpyDeviceToHost), "cudaMemcpy D->H");
  
  printf("Sum of the array computed in GPU is %d\nTotal Time taken on GPU: %fs\n", h_sum, time_elapsed_gpu/1000);

  //release GPU memory
  cudaFree(d_data);
  cudaFree(d_sum);
  free(h_data);

  return 0;
}

Overwriting /content/q2part1.cu


In [90]:
!nvcc /content/q2part1.cu -lcurand -o q2part1 && ./q2part1

Sum of the array computed at CPU is 100
Total Time taken on CPU: 0.000002s
Sum of the array computed in GPU is 100
Total Time taken on GPU: 0.000030s
